In [1]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv("churn-5000.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)


#BOX-COX
#lemda=0.5
#num_df=(num_df**lemda)
#num_df=num_df-1
#num_df[num_df < 0]=0
#num_df=num_df.div(lemda)
#End BOX-COX
#Z-score
#num_df=(num_df-num_df.min())/(num_df.std(ddof=0)) ##Z-score

#Log
num_df=round(np.log(num_df.add(1)),2)
num_df=num_df.replace([np.inf, -np.inf], np.nan)
#End Log

num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn


In [3]:
X.shape

(5000, 19)

In [4]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
17,international_plan,303.963357
2,number_vmail_messages,144.279348
18,voice_mail_plan,45.058340
15,number_customer_service_calls,38.021508
13,total_intl_calls,2.181351
16,state,2.033226
5,total_day_charge,1.716429
3,total_day_minutes,1.584154
12,total_intl_minutes,1.416559
6,total_eve_minutes,0.861565


In [5]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(5000, 15)

In [6]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

4024 269 509 198
pod:  0.28005657708628007
pof:  0.06266014442115071
AUC:  0.6086982163325647


In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.6s


{'var_smoothing': 0.0006579332246575676}
GaussianNB(priors=None, var_smoothing=0.0006579332246575676)
0.8625333333333334


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   10.0s finished


In [8]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.0006579332246575676)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4252 41 639 68
pod:  0.09618104667609619
pof:  0.009550430934078733
AUC:  0.5433153078710087
accuracy:  0.864


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 345 out of 360 | elapsed:   11.8s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   11.9s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


{'C': 7, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'lbfgs'}
LogisticRegression(C=7, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
0.8638


In [10]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=7, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", Convergenc

4228 65 612 95
pod:  0.13437057991513437
pof:  0.015140927090612625
AUC:  0.5596148264122609
accuracy:  0.8646


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=11, p=2,
           weights='distance')
0.8602


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    8.6s finished


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [15]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance' )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4236 57 653 54
pod:  0.07637906647807638
pof:  0.013277428371767994
AUC:  0.5315508190531542
accuracy:  0.858


In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4247 46 681 26
pod:  0.036775106082036775
pof:  0.010715117633356627
AUC:  0.51302999422434
accuracy:  0.8546


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  4.2min finished


{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=11, p=2,
           weights='distance')
0.8602


In [17]:
print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

{'bootstrap': True, 'max_depth': 80, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 1000}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.9442


In [18]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier( bootstrap=True, class_weight=None, criterion='gini',
            max_depth=80, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4256 37 236 471
pod:  0.6661951909476662
pof:  0.008618681574656418
AUC:  0.8287882546865049
accuracy:  0.9454


In [19]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


4246 47 260 447
pod:  0.6322489391796322
pof:  0.010948054973212206
AUC:  0.81065044210321


In [20]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4064 229 213 494
pod:  0.6987270155586988
pof:  0.05334265082692756
AUC:  0.8226921823658856
accuracy:  0.9116


In [21]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4051 242 208 499
pod:  0.7057991513437057
pof:  0.056370836245050085
AUC:  0.8247141575493279


In [22]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4239 54 224 483
pod:  0.6831683168316832
pof:  0.012578616352201259
AUC:  0.835294850239741
accuracy:  0.9444


In [23]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4242 51 221 486
pod:  0.6874115983026874
pof:  0.011879804332634521
AUC:  0.8377658969850265
accuracy:  0.9456


In [24]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 147us/step - loss: 0.4381 - acc: 0.8558 - val_loss: 0.4085 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 60us/step - loss: 0.4116 - acc: 0.8586 - val_loss: 0.4043 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.4067 - acc: 0.8586 - val_loss: 0.4088 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.4038 - acc: 0.8586 - val_loss: 0.4027 - val_acc: 0.8589
Epoch 5/30
3599/3599 [==============================] - 0s 52us/step - loss: 0.4046 - acc: 0.8586 - val_loss: 0.4017 - val_acc: 0.8589
Epoch 6/30
3599/3599 [==============================] - 0s 52us/step - loss: 0.3912 - acc: 0.8586 - val_loss: 0.3859 - val_acc: 0.8589
Epoch 7/30
3599/3599 [

 [0.]]
pod 1st:  0.0
Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 170us/step - loss: 0.4267 - acc: 0.8550 - val_loss: 0.4100 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 57us/step - loss: 0.4157 - acc: 0.8586 - val_loss: 0.4157 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 57us/step - loss: 0.4053 - acc: 0.8586 - val_loss: 0.4009 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 50us/step - loss: 0.4007 - acc: 0.8586 - val_loss: 0.3918 - val_acc: 0.8589
Epoch 5/30
3599/3599 [==============================] - 0s 53us/step - loss: 0.3962 - acc: 0.8586 - val_loss: 0.3860 - val_acc: 0.8589
Epoch 6/30
3599/3599 [==============================] - 0s 55us/step - loss: 0.3960 - acc: 0.8586 - val_loss: 0.3868 - val_acc: 0.8589
Epoch 7/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.3923 - acc: 0.8588 - val_loss: 0.3893 - va

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 173us/step - loss: 0.4281 - acc: 0.8530 - val_loss: 0.4039 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 50us/step - loss: 0.4063 - acc: 0.8586 - val_loss: 0.3920 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.3946 - acc: 0.8586 - val_loss: 0.3925 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 51us/step - loss: 0.3978 - acc: 0.8586 - val_loss: 0.3817 - val_acc: 0.8589
Epoch 5/30
3599/3599 [==============================] - 0s 50us/step - loss: 0.3889 - acc: 0.8586 - val_loss: 0.3759 - val_acc: 0.8589
Epoch 6/30
3599/3599 [==============================] - 0s 53us/step - loss: 0.3885 - acc: 0.8586 - val_loss: 0.3729 - val_acc: 0.8589
Epoch 7/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.3842 - acc: 0.8586 - val_loss: 0.3683 - val_acc: 0.8589
Epoch 8

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 192us/step - loss: 0.4185 - acc: 0.8558 - val_loss: 0.4074 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 47us/step - loss: 0.4081 - acc: 0.8586 - val_loss: 0.4059 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 51us/step - loss: 0.4020 - acc: 0.8586 - val_loss: 0.3894 - val_acc: 0.8589
Epoch 4/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.3955 - acc: 0.8586 - val_loss: 0.3814 - val_acc: 0.8589
Epoch 5/30
3600/3600 [==============================] - 0s 52us/step - loss: 0.3860 - acc: 0.8589 - val_loss: 0.3770 - val_acc: 0.8600
Epoch 6/30
3600/3600 [==============================] - 0s 52us/step - loss: 0.3835 - acc: 0.8592 - val_loss: 0.3668 - val_acc: 0.8589
Epoch 7/30
3600/3600 [==============================] - 0s 52us/step - loss: 0.3783 - acc: 0.8569 - val_loss: 0.3676 - val_acc: 0.8600
Epoch 8

...

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 261us/step - loss: 0.4497 - acc: 0.8514 - val_loss: 0.4095 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 55us/step - loss: 0.4174 - acc: 0.8586 - val_loss: 0.4034 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.4049 - acc: 0.8586 - val_loss: 0.3957 - val_acc: 0.8589
Epoch 4/30
3600/3600 [==============================] - 0s 54us/step - loss: 0.4059 - acc: 0.8586 - val_loss: 0.4046 - val_acc: 0.8589
Epoch 5/30
3600/3600 [==============================] - 0s 54us/step - loss: 0.3985 - acc: 0.8586 - val_loss: 0.3836 - val_acc: 0.8589
Epoch 6/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.3918 - acc: 0.8586 - val_loss: 0.3813 - val_acc: 0.8589
Epoch 7/30
3600/3600 [==============================] - 0s 55us/step - loss: 0.3878 - acc: 0.8586 - val_loss: 0.3723 - val_acc: 0.8589
Epoch 8

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 270us/step - loss: 0.4306 - acc: 0.8578 - val_loss: 0.4130 - val_acc: 0.8590
Epoch 2/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.4142 - acc: 0.8583 - val_loss: 0.4055 - val_acc: 0.8590
Epoch 3/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.4089 - acc: 0.8583 - val_loss: 0.4024 - val_acc: 0.8590
Epoch 4/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.4042 - acc: 0.8583 - val_loss: 0.4111 - val_acc: 0.8590
Epoch 5/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.4018 - acc: 0.8583 - val_loss: 0.4025 - val_acc: 0.8590
Epoch 6/30
3600/3600 [==============================] - 0s 65us/step - loss: 0.3943 - acc: 0.8583 - val_loss: 0.4018 - val_acc: 0.8590
Epoch 7/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.3951 - acc: 0.8583 - val_loss: 0.3877 - val_acc: 0.8590
Epoch 8

 [1.]]
pod 1st:  0.07142857142857142
Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 290us/step - loss: 0.4255 - acc: 0.8506 - val_loss: 0.3922 - val_acc: 0.8646
Epoch 2/30
3600/3600 [==============================] - 0s 72us/step - loss: 0.4062 - acc: 0.8569 - val_loss: 0.4016 - val_acc: 0.8646
Epoch 3/30
3600/3600 [==============================] - 0s 70us/step - loss: 0.4032 - acc: 0.8569 - val_loss: 0.3780 - val_acc: 0.8646
Epoch 4/30
3600/3600 [==============================] - 0s 68us/step - loss: 0.4045 - acc: 0.8572 - val_loss: 0.3868 - val_acc: 0.8646
Epoch 5/30
3600/3600 [==============================] - 0s 73us/step - loss: 0.3916 - acc: 0.8569 - val_loss: 0.3715 - val_acc: 0.8646
Epoch 6/30
3600/3600 [==============================] - 0s 61us/step - loss: 0.3897 - acc: 0.8569 - val_loss: 0.3676 - val_acc: 0.8646
Epoch 7/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.3788 - acc: 0.8583 - val_l

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 304us/step - loss: 0.4383 - acc: 0.8519 - val_loss: 0.4102 - val_acc: 0.8646
Epoch 2/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.4164 - acc: 0.8569 - val_loss: 0.3973 - val_acc: 0.8646
Epoch 3/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.4044 - acc: 0.8569 - val_loss: 0.3885 - val_acc: 0.8646
Epoch 4/30
3600/3600 [==============================] - 0s 69us/step - loss: 0.3990 - acc: 0.8569 - val_loss: 0.3926 - val_acc: 0.8646
Epoch 5/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.4003 - acc: 0.8569 - val_loss: 0.3909 - val_acc: 0.8646
Epoch 6/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.3895 - acc: 0.8569 - val_loss: 0.3792 - val_acc: 0.8646
Epoch 7/30
3600/3600 [==============================] - 0s 62us/step - loss: 0.3871 - acc: 0.8569 - val_loss: 0.3741 - val_acc: 0.8646
Epoch 8

In [25]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 3599 samples, validate on 900 samples
Epoch 1/5
3599/3599 [==============================] - 2s 449us/step - loss: 0.4788 - acc: 0.8413 - val_loss: 0.4044 - val_acc: 0.8589
Epoch 2/5
3599/3599 [==============================] - 0s 76us/step - loss: 0.4057 - acc: 0.8586 - val_loss: 0.4113 - val_acc: 0.8589
Epoch 3/5
3599/3599 [==============================] - 0s 77us/step - loss: 0.4013 - acc: 0.8586 - val_loss: 0.3986 - val_acc: 0.8589
Epoch 4/5
3599/3599 [==============================] - 0s 80us/step - loss: 0.3993 - acc: 0.8586 - val_loss: 0.4119 - val_acc: 0.8589
Epoch 5/5
3599/3599 [==============================] - 0s 75us/step - loss: 0.3942 - acc: 0.8586 - val_loss: 0.4202 - val_acc: 0.8589
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 3600 samples, validate on 900 samples
Epoch 1/5
3600/3600 [==============================] - 3s 714us/step - loss: 0.4928 - acc: 0.8500 - val_loss: 0.4054 - val_acc: 0.8589
Epoch 2/5
3600/3600 [==============================] - 0s 91us/step - loss: 0.4057 - acc: 0.8586 - val_loss: 0.4154 - val_acc: 0.8589
Epoch 3/5
3600/3600 [==============================] - 0s 90us/step - loss: 0.4013 - acc: 0.8586 - val_loss: 0.4154 - val_acc: 0.8589
Epoch 4/5
3600/3600 [==============================] - 0s 100us/step - loss: 0.3990 - acc: 0.8586 - val_loss: 0.3984 - val_acc: 0.8589
Epoch 5/5
3600/3600 [==============================] - 0s 88us/step - loss: 0.3949 - acc: 0.8586 - val_loss: 0.4289 - val_acc: 0.8589
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]


y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]